In [1]:
import os

In [2]:
!pwd

/d/PythonProjects/Text-summarizer/research


In [3]:
os.chdir('../')

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass
class DataIngetionConfig:
    root_dir:Path 
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [5]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories, get_size

In [37]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
        
    def get_data_ingetion_config(self)-> DataIngetionConfig:
        
        config=self.config.data_ingetion
        
        create_directories([config.root_dir])
        data_ingetion_config=DataIngetionConfig(
            root_dir=config.root_dir,
            source_url=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingetion_config        


In [38]:
from urllib import request
from zipfile import ZipFile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [42]:
class DataIngetion:
    def __init__(self,
                 config:DataIngetionConfig):
        
        self.config=config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header=request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
            
        
    def unzip_data(self):
        unzip_dir=self.config.unzip_dir
        os.makedirs(unzip_dir,exist_ok=True)
        with ZipFile(self.config.local_data_file,'r') as data_file:
            data_file.extractall(unzip_dir)    


In [43]:
try:
    config=ConfigurationManager()
    data_ingetion_config=config.get_data_ingetion_config()
    data_ingetion=DataIngetion(data_ingetion_config)
    data_ingetion.download_data()
    data_ingetion.unzip_data()
except Exception as e:
    raise e

[2023-08-02 18:03:43,370 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2023-08-02 18:03:43,372 : INFO : common : yaml file: params.yaml loaded successfully]
[2023-08-02 18:03:43,373 : INFO : common : created directory at: artifacts]
[2023-08-02 18:03:43,374 : INFO : common : created directory at: artifacts/data_ingestion]
[2023-08-02 18:03:43,375 : INFO : 1059116385 : File already exists of size: ~ 7718 KB]


In [28]:
data_ingetion_config.local_data_file

'artifacts/data_ingetion/summarizer-data.zip'